# ОБУЧЕНИЕ

In [40]:
from catboost import CatBoostRegressor, Pool
from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import LabelEncoder
import pandas as pd
import polars as pl
import xgboost as xgb
import optuna
import numpy as np
from sklearn.ensemble import VotingRegressor

In [6]:
train = pl.read_parquet('/kaggle/input/fin-fin-vtb/train_final_final_full.parquet')

In [7]:
train

inn_id,week,target,month,year,week_of_month,quarter,holidays_in_week,weekends_in_week,month_week,ipul,id_region_f,id_region_l,id_region_u,main_okved_group,diff_datopen_report_date_flg,diff_datopen_report_date_flg_min,out_vtb_trns_count_w,out_vtb_date_w,out_vtb_cinn_w,out_vtb_trns_class_encoded_w,out_vtb_doc_payer_bank_name_encoded_w,outl_vtb_trns_count_w,in_vtb_trns_count_w,in_vtb_date_w,in_vtb_cinn_w,in_vtb_trns_class_encoded_w,in_vtb_doc_payee_bank_name_encoded_w,inl_vtb_trns_count_w,week_avg_amount_inn_1,week_median_amount_inn_1,week_p90_amount_inn_1,week_p75_amount_inn_1,week_p5_amount_inn_1,week_p25_amount_inn_1,week_p10_amount_inn_1,week_large_trns_count_inn_1,…,week_of_month_large_trns_count_full,week_of_month_small_trns_count_full,week_of_month_large_trns_p_full,week_of_month_small_trns_p_full,week_of_month_avg_amount_full_1,week_of_month_median_amount_full_1,week_of_month_p90_amount_full_1,week_of_month_p75_amount_full_1,week_of_month_p5_amount_full_1,week_of_month_p25_amount_full_1,week_of_month_p10_amount_full_1,week_of_month_large_trns_count_full_1,week_of_month_small_trns_count_full_1,week_of_month_large_trns_p_full_1,week_of_month_small_trns_p_full_1,month_avg_amount_full,month_median_amount_full,month_p90_amount_full,month_p75_amount_full,month_p5_amount_full,month_p25_amount_full,month_p10_amount_full,month_large_trns_count_full,month_small_trns_count_full,month_large_trns_p_full,month_small_trns_p_full,month_avg_amount_full_1,month_median_amount_full_1,month_p90_amount_full_1,month_p75_amount_full_1,month_p5_amount_full_1,month_p25_amount_full_1,month_p10_amount_full_1,month_large_trns_count_full_1,month_small_trns_count_full_1,month_large_trns_p_full_1,month_small_trns_p_full_1
str,i16,f64,i8,i32,i8,i8,i8,i64,str,i8,str,str,u32,str,f64,f64,f64,u32,u32,u32,u32,f64,f64,u32,u32,u32,u32,f64,f64,f64,f64,f64,f64,f64,f64,u32,…,u32,u32,f64,f64,f64,f64,f64,f64,f64,f64,f64,u32,u32,f64,f64,f64,f64,f64,f64,f64,f64,f64,u32,u32,f64,f64,f64,f64,f64,f64,f64,f64,f64,u32,u32,f64,f64
"""inn3521926""",10,8.832493,10,2022,1,4,0,2,"""10-1""",0,"""45""","""45""",1,"""47""",144.0,96.0,678.0,315,66,3,1,1.0,621.0,330,119,6,3,1.0,null,null,null,null,null,null,null,null,…,0,478260,0.0,1.0,null,null,null,null,null,null,null,null,null,null,null,12.285946,13.100494,15.443621,14.416876,13.100495,11.644328,9.491425,0,621738,0.0,1.0,null,null,null,null,null,null,null,null,null,null,null
"""inn4139002""",10,10.914028,10,2022,1,4,0,2,"""10-1""",1,"""36""","""36""",1,"""74""",96.0,96.0,919.0,343,55,10,1,1.0,462.0,143,27,13,2,1.0,null,null,null,null,null,null,null,null,…,0,478260,0.0,1.0,null,null,null,null,null,null,null,null,null,null,null,12.285946,13.100494,15.443621,14.416876,13.100495,11.644328,9.491425,0,621738,0.0,1.0,null,null,null,null,null,null,null,null,null,null,null
"""inn3570711""",10,13.761011,10,2022,1,4,0,2,"""10-1""",1,"""20""","""20""",1,"""43""",96.0,96.0,1439.0,416,210,10,1,1.0,541.0,284,116,14,3,1.0,null,null,null,null,null,null,null,null,…,0,478260,0.0,1.0,null,null,null,null,null,null,null,null,null,null,null,12.285946,13.100494,15.443621,14.416876,13.100495,11.644328,9.491425,0,621738,0.0,1.0,null,null,null,null,null,null,null,null,null,null,null
"""inn4167881""",10,11.396917,10,2022,1,4,0,2,"""10-1""",1,"""36""","""36""",1,"""43""",144.0,144.0,866.0,370,147,5,1,1.0,486.0,253,46,6,1,1.0,null,null,null,null,null,null,null,null,…,0,478260,0.0,1.0,null,null,null,null,null,null,null,null,null,null,null,12.285946,13.100494,15.443621,14.416876,13.100495,11.644328,9.491425,0,621738,0.0,1.0,null,null,null,null,null,null,null,null,null,null,null
"""inn4387940""",10,13.774928,10,2022,1,4,0,2,"""10-1""",1,"""40""","""40""",1,"""47""",192.0,192.0,3076.0,572,149,7,2,1.0,22694.0,612,76,12,3,24.0,null,null,null,null,null,null,null,null,…,0,478260,0.0,1.0,null,null,null,null,null,null,null,null,null,null,null,12.285946,13.100494,15.443621,14.416876,13.100495,11.644328,9.491425,0,621738,0.0,1.0,null,null,null,null,null,null,null,null,nul

In [8]:
val =  train.filter((pl.col('week') < 118) & (pl.col('week') >= 106)).to_pandas()
train_part = train.filter(pl.col('week') < 106).to_pandas()

In [9]:
cat_cols = ['inn_id', 'month', 'year', 'week_of_month', 'quarter', 'ipul', 'id_region_f', 'id_region_l', 'id_region_u', 'main_okved_group', 'diff_datopen_report_date_flg', 'diff_datopen_report_date_flg_min', 'month_week']

In [28]:
y_val = val['target']
X_val = val.drop(['target'], axis=1)
X_val[cat_cols] = X_val[cat_cols].fillna('missing').astype(str)

y_train_part = train_part['target']
X_train_part = train_part.drop(['target'], axis=1)
X_train_part[cat_cols] = X_train_part[cat_cols].fillna('missing').astype(str)

Работа с категориальными признаками

In [11]:
for col in cat_cols:
    X_train_part[col], _ = pd.factorize(X_train_part[col].astype(str))
    X_val[col], _ = pd.factorize(X_val[col].astype(str))

# Я решил сделать ансамбль из двух момделей Xgboost и CatBoost и прикрутить туда Optuna

## Optuna для XGBoost

In [22]:
def obective(trial):
    X_train_xgb = xgb.DMatrix(X_train_part, label=y_train_part)
    X_val_xgb = xgb.DMatrix(X_val, label=y_val)
    params = {
        'objective': 'reg:squarederror',
        'eval_metric': 'rmse',
        'max_depth': trial.suggest_int('max_depth', 6, 8),
        'learning_rate': trial.suggest_float('learning_rate', 0.01, 0.1),
        'tree_method': 'hist',
        'device': 'cuda:0',
        'alpha': 0,
        'lambda': 1,
        'min_child_weight': trial.suggest_float('min_child_weight', 0.1, 1),
        'colsample_bytree': trial.suggest_float('colsample_bytree', 0.3, 0.8)
    }
    num_boost_round = trial.suggest_int('num_boost_rounds', 50, 500)
    evals = [(X_train_xgb, 'train'), (X_val_xgb, 'eval')]
    model = xgb.train(params, X_train_xgb,
                      num_boost_round=num_boost_round,
                     evals=evals,
                     early_stopping_rounds=20)
    
    pred = model.predict(X_val_xgb)
    rmse = np.sqrt(mean_squared_error(y_val, pred))
    return rmse
    

In [23]:
study = optuna.create_study(
    direction='minimize',
    study_name='lightgbm_regression',
    sampler=optuna.samplers.TPESampler(),
    pruner=optuna.pruners.MedianPruner(),
    load_if_exists=True
)

[I 2025-04-05 19:00:39,122] A new study created in memory with name: lightgbm_regression


In [ ]:
study.optimize(obective, n_trials=50)

In [27]:
xgb_best_params = study.best_params
xgb_best_values = study.best_value
print(f'Наилучшие параметры: {xgb_best_params}')
print(f'Наилучшая rmse: {xgb_best_values}')

Наилучшие параметры: {'max_depth': 6, 'learning_rate': 0.03747182457350956, 'min_child_weight': 0.5714383795404899, 'colsample_bytree': 0.35631564612788225, 'num_boost_rounds': 352}
Наилучшая rmse: 2.5266239832626987


## OPtuna для CatBoosting

In [35]:
def cat_objective(trial):
    pool_train_part = Pool(data=X_train_part, label=y_train_part, cat_features=cat_cols)
    pool_val = Pool(data=X_val, label=y_val, cat_features=cat_cols)
    model = CatBoostRegressor(
        iterations=trial.suggest_int('iterations', 1000, 3000),
        learning_rate=trial.suggest_float('learning_rate', 0.01, 0.3),
        loss_function='RMSE',
        eval_metric='RMSE',
        l2_leaf_reg=trial.suggest_float('l2_leaf_reg', 1, 10),
        bootstrap_type='MVS',
        subsample=trial.suggest_float('subsample', 0.5, 1),
        early_stopping_rounds=20,
        task_type='GPU',
        verbose=100,
    )
        
    model.fit(pool_train_part, eval_set=pool_val, verbose=100)
    pred = model.predict(pool_val)
    rmse = np.sqrt(mean_squared_error(y_val, pred))
    return rmse

In [36]:
study_cat = optuna.create_study(
    direction='minimize',
    study_name='catboost_regression',
    sampler=optuna.samplers.TPESampler(),
    pruner=optuna.pruners.MedianPruner(),
    load_if_exists=True
)

[I 2025-04-05 19:47:55,882] A new study created in memory with name: catboost_regression


In [ ]:
study_cat.optimize(cat_objective, n_trials=50)

In [38]:
cat_best_params = study_cat.best_params
cat_best_values = study_cat.best_value
print(f'Наилучшие параметры: {cat_best_params}')
print(f'Наилучшая rmse: {cat_best_values}')

Наилучшие параметры: {'iterations': 1864, 'learning_rate': 0.044149929211601255, 'l2_leaf_reg': 8.035511509270492, 'subsample': 0.7242205709760727}
Наилучшая rmse: 2.859365123400248


# Создание ансамбля(подбор гиперпараметров)

## Данные

In [72]:
for col in cat_cols:
    X_train_part[col], _ = pd.factorize(X_train_part[col].astype(str))
    X_val[col], _ = pd.factorize(X_val[col].astype(str))

voting_X_train = X_train_part
voting_y_train = y_train_part

## Инициализация моделей

In [73]:
def voiting_objective(trial):
    xgb_model = xgb.XGBRegressor(
        objective='reg:squarederror',
        eval_metric='rmse',
        max_depth=trial.suggest_int('max_depth', 6, 8),
        learning_rate=trial.suggest_float('xgb_learning_rate', 0.01, 0.1),
        tree_method='hist',
        device='cuda:0',
        alpha=0,
        reg_lambda=1,
        min_child_weight=trial.suggest_float('min_child_weight', 0.1, 1),
        colsample_bytree=trial.suggest_float('colsample_bytree', 0.3, 0.8)
    )
    
    cat_model = CatBoostRegressor(
        iterations=trial.suggest_int('iterations', 1000, 3000),
        learning_rate=trial.suggest_float('cat_learning_rate', 0.01, 0.3),
        loss_function='RMSE',
        eval_metric='RMSE',
        l2_leaf_reg=trial.suggest_float('l2_leaf_reg', 1, 10),
        bootstrap_type='MVS',
        subsample=trial.suggest_float('subsample', 0.5, 1),
        early_stopping_rounds=20,
        task_type='GPU',
        verbose=100,
    )

    voting_regressor = VotingRegressor(estimators=[('xgb', xgb_model),
                                                  ('cat_boost', cat_model)])
    voting_regressor.fit(voting_X_train, voting_y_train)
    prediction = voting_regressor.predict(X_val)
    rmse = np.sqrt(mean_squared_error(y_val, prediction))
    
    return rmse

In [74]:
voting_study = optuna.create_study(direction='minimize',
    study_name='voting_regressor optimize',
    sampler=optuna.samplers.TPESampler(),
    pruner=optuna.pruners.MedianPruner(),
    load_if_exists=True
)

[I 2025-04-05 21:26:44,391] A new study created in memory with name: voting_regressor optimize


In [ ]:
voting_study.optimize(voiting_objective, n_trials=100)

0:	learn: 3.7047069	total: 14.2ms	remaining: 29.5s
100:	learn: 2.2901287	total: 1.26s	remaining: 24.6s
200:	learn: 2.2606113	total: 2.49s	remaining: 23.2s
300:	learn: 2.2522801	total: 3.69s	remaining: 21.8s
400:	learn: 2.2486336	total: 4.93s	remaining: 20.6s
500:	learn: 2.2485855	total: 6.09s	remaining: 19.1s
600:	learn: 2.2485375	total: 7.25s	remaining: 17.8s
700:	learn: 2.2485273	total: 8.41s	remaining: 16.5s
800:	learn: 2.2485074	total: 9.58s	remaining: 15.2s
900:	learn: 2.2484993	total: 10.7s	remaining: 14s
1000:	learn: 2.2484934	total: 11.9s	remaining: 12.8s
1100:	learn: 2.2484903	total: 13.1s	remaining: 11.6s
1200:	learn: 2.2484885	total: 14.2s	remaining: 10.3s
1300:	learn: 2.2484858	total: 15.4s	remaining: 9.14s
1400:	learn: 2.2484839	total: 16.6s	remaining: 7.95s
1500:	learn: 2.2484827	total: 17.7s	remaining: 6.77s
1600:	learn: 2.2484815	total: 18.9s	remaining: 5.58s
1700:	learn: 2.2484809	total: 20s	remaining: 4.39s
1800:	learn: 2.2484808	total: 21.2s	remaining: 3.21s
1900:	le

[I 2025-04-05 21:27:41,740] Trial 0 finished with value: 2.5546240469825046 and parameters: {'max_depth': 7, 'xgb_learning_rate': 0.06865617208375763, 'min_child_weight': 0.8543515918963404, 'colsample_bytree': 0.6041537592222245, 'iterations': 2074, 'cat_learning_rate': 0.07161018802640909, 'l2_leaf_reg': 1.7736015676497492, 'subsample': 0.771341497087695}. Best is trial 0 with value: 2.5546240469825046.


0:	learn: 3.3073726	total: 14.9ms	remaining: 15.8s
100:	learn: 2.2529264	total: 1.25s	remaining: 11.9s
200:	learn: 2.2514051	total: 2.5s	remaining: 10.8s
300:	learn: 2.2513918	total: 3.79s	remaining: 9.64s
400:	learn: 2.2513786	total: 5.08s	remaining: 8.42s
500:	learn: 2.2513717	total: 6.36s	remaining: 7.17s
600:	learn: 2.2513701	total: 7.64s	remaining: 5.91s
700:	learn: 2.2513689	total: 8.96s	remaining: 4.66s
800:	learn: 2.2513688	total: 10.3s	remaining: 3.39s
900:	learn: 2.2513676	total: 11.5s	remaining: 2.11s
1000:	learn: 2.2513675	total: 12.8s	remaining: 832ms
1065:	learn: 2.2513675	total: 13.7s	remaining: 0us


[I 2025-04-05 21:28:29,919] Trial 1 finished with value: 2.5730165126750832 and parameters: {'max_depth': 8, 'xgb_learning_rate': 0.023483214594132808, 'min_child_weight': 0.32934127562095744, 'colsample_bytree': 0.6574331158101431, 'iterations': 1066, 'cat_learning_rate': 0.27645079778291115, 'l2_leaf_reg': 3.888476944892702, 'subsample': 0.6534379342573735}. Best is trial 0 with value: 2.5546240469825046.


0:	learn: 3.5454616	total: 14.2ms	remaining: 27.9s
100:	learn: 2.2620920	total: 1.24s	remaining: 23s
200:	learn: 2.2579949	total: 2.42s	remaining: 21.3s
300:	learn: 2.2579084	total: 3.6s	remaining: 20s
400:	learn: 2.2578848	total: 4.78s	remaining: 18.7s
500:	learn: 2.2578778	total: 5.96s	remaining: 17.5s
600:	learn: 2.2578745	total: 7.15s	remaining: 16.3s
700:	learn: 2.2578723	total: 8.34s	remaining: 15.1s
800:	learn: 2.2578713	total: 9.52s	remaining: 13.9s
900:	learn: 2.2578710	total: 10.7s	remaining: 12.7s
1000:	learn: 2.2578711	total: 11.9s	remaining: 11.5s
1100:	learn: 2.2578712	total: 13s	remaining: 10.3s
1200:	learn: 2.2578716	total: 14.2s	remaining: 9.12s
1300:	learn: 2.2578713	total: 15.4s	remaining: 7.93s
1400:	learn: 2.2578710	total: 16.6s	remaining: 6.75s
1500:	learn: 2.2578709	total: 17.8s	remaining: 5.56s
1600:	learn: 2.2578707	total: 18.9s	remaining: 4.38s
1700:	learn: 2.2578705	total: 20.1s	remaining: 3.19s
1800:	learn: 2.2578705	total: 21.3s	remaining: 2.01s
1900:	learn

...

[I 2025-04-05 21:34:59,850] Trial 8 finished with value: 2.5713431245157516 and parameters: {'max_depth': 8, 'xgb_learning_rate': 0.06419723880313959, 'min_child_weight': 0.49727902828834436, 'colsample_bytree': 0.4857276266802206, 'iterations': 1301, 'cat_learning_rate': 0.12978733002036594, 'l2_leaf_reg': 1.4100072714688543, 'subsample': 0.5711024923796946}. Best is trial 3 with value: 2.5457618875664574.


0:	learn: 3.4917051	total: 14.1ms	remaining: 20s
100:	learn: 2.2549071	total: 1.24s	remaining: 16.2s
200:	learn: 2.2474491	total: 2.43s	remaining: 14.8s
300:	learn: 2.2455579	total: 3.64s	remaining: 13.6s
400:	learn: 2.2424304	total: 4.84s	remaining: 12.3s
500:	learn: 2.2422277	total: 6.04s	remaining: 11.1s
600:	learn: 2.2422136	total: 7.23s	remaining: 9.88s
700:	learn: 2.2422089	total: 8.42s	remaining: 8.66s
800:	learn: 2.2422002	total: 9.62s	remaining: 7.46s
900:	learn: 2.2421951	total: 10.8s	remaining: 6.25s
1000:	learn: 2.2421953	total: 12s	remaining: 5.04s
1100:	learn: 2.2421938	total: 13.2s	remaining: 3.84s
1200:	learn: 2.2421945	total: 14.4s	remaining: 2.64s
1300:	learn: 2.2421943	total: 15.5s	remaining: 1.45s
1400:	learn: 2.2421942	total: 16.7s	remaining: 251ms
1421:	learn: 2.2421942	total: 17s	remaining: 0us


[I 2025-04-05 21:35:47,596] Trial 9 finished with value: 2.5601505574814616 and parameters: {'max_depth': 6, 'xgb_learning_rate': 0.07769874564836304, 'min_child_weight': 0.9291244106921792, 'colsample_bytree': 0.5034972654656295, 'iterations': 1422, 'cat_learning_rate': 0.17832534693898974, 'l2_leaf_reg': 8.412570462315813, 'subsample': 0.5928978987986449}. Best is trial 3 with value: 2.5457618875664574.


0:	learn: 3.7993755	total: 14ms	remaining: 35.9s
100:	learn: 2.3684008	total: 1.3s	remaining: 31.6s
200:	learn: 2.3050125	total: 2.53s	remaining: 29.7s
300:	learn: 2.2848451	total: 3.77s	remaining: 28.3s
400:	learn: 2.2718286	total: 5.02s	remaining: 27.1s
500:	learn: 2.2630327	total: 6.25s	remaining: 25.7s
600:	learn: 2.2564192	total: 7.48s	remaining: 24.4s
700:	learn: 2.2515342	total: 8.72s	remaining: 23.2s
800:	learn: 2.2471218	total: 9.95s	remaining: 21.9s
900:	learn: 2.2445651	total: 11.2s	remaining: 20.6s
1000:	learn: 2.2415000	total: 12.4s	remaining: 19.3s
1100:	learn: 2.2394055	total: 13.6s	remaining: 18.1s
1200:	learn: 2.2374554	total: 14.8s	remaining: 16.8s
1300:	learn: 2.2359408	total: 16s	remaining: 15.5s
1400:	learn: 2.2345211	total: 17.2s	remaining: 14.3s
1500:	learn: 2.2333107	total: 18.5s	remaining: 13.1s
1600:	learn: 2.2327900	total: 19.7s	remaining: 11.8s
1700:	learn: 2.2327135	total: 20.9s	remaining: 10.6s
1800:	learn: 2.2326874	total: 22.1s	remaining: 9.33s
1900:	lea

[I 2025-04-05 21:36:50,210] Trial 10 finished with value: 2.567439164121298 and parameters: {'max_depth': 7, 'xgb_learning_rate': 0.09987333227099687, 'min_child_weight': 0.6959798900539502, 'colsample_bytree': 0.7866523556199138, 'iterations': 2563, 'cat_learning_rate': 0.026234235541708117, 'l2_leaf_reg': 4.665878145319397, 'subsample': 0.5018451293318518}. Best is trial 3 with value: 2.5457618875664574.


In [ ]:
voting_study = voting_study.best_params
voting_study = voting_study.best_value
print(f'Наилучшие параметры: {voting_study}')
print(f'Наилучшая rmse: {voting_study}')

In [ ]:
y_pred = model.predict(X_val)
y_pred = np.maximum(0, y_pred)
msle = mean_squared_error(y_val, y_pred)
msle

In [ ]:
pd.set_option('display.max_rows', None)
importance_df = pd.DataFrame({
    'Feature': X_val.columns,
    'Importance': model.get_feature_importance()
}).sort_values('Importance', ascending=False)
importance_df.head(100)

,Feature,Importance
72,month_avg_amount_inn,39.375218
17,out_vtb_date_w,9.299673
77,month_p25_amount_inn,7.758012
78,month_p10_amount_inn,7.364571
32,week_p5_amount_inn_1,3.744885
75,month_p75_amount_inn,3.387154
29,week_median_amount_inn_1,3.265926
0,inn_id,2.777796
33,week_p25_amount_inn_1,2.725841
28,week_avg_amount_inn_1,2.525159


In [ ]:
test = train.filter(pl.col('week') >= 118).to_pandas()
train_full = train.filter(pl.col('week') < 118).to_pandas()

In [ ]:
X_test = test.drop(['target'], axis=1)
X_test[cat_cols] = X_test[cat_cols].fillna('missing').astype(str)


y_train_full = train_full['target']
X_train_full = train_full.drop(['target'], axis=1)
X_train_full[cat_cols] = X_train_full[cat_cols].fillna('missing').astype(str)

pool_train_full = Pool(data=X_train_full, label=y_train_full, cat_features=cat_cols)

In [ ]:
#Полученные в optuna
perfect_params = {'iterations': 100,
                  'learning_rate':0.01,
                  'loss_function':'RMSE',
                  'eval_metric':'RMSE',
                  'verbose':100,
                  'task_type':'GPU'}

In [ ]:
model_final = CatBoostRegressor(**perfect_params)

In [ ]:
model_final.fit(pool_train_full)

0:	learn: 3.0134925	total: 13.9ms	remaining: 1.38s
99:	learn: 2.2757299	total: 1.21s	remaining: 0us


In [ ]:
import pandas as pd
pd.set_option('display.max_rows', None)
importance_df = pd.DataFrame({
    'Feature': X_val.columns,
    'Importance': model_final.get_feature_importance()
}).sort_values('Importance', ascending=False)
importance_df.head(100)

,Feature,Importance
72,month_avg_amount_inn,40.413247
77,month_p25_amount_inn,11.143209
29,week_median_amount_inn_1,6.426131
32,week_p5_amount_inn_1,5.683871
94,target_1,4.052070
76,month_p5_amount_inn,3.468197
33,week_p25_amount_inn_1,3.442977
34,week_p10_amount_inn_1,3.170021
28,week_avg_amount_inn_1,3.164770
30,week_p90_amount_inn_1,2.490689


In [ ]:
result = np.exp(model_final.predict(X_test)) #тут обязательно экспоненту

In [ ]:
submit = X_test[['inn_id', 'week']]
submit['predict'] = result
submit.to_csv('sumbmit_example.csv')

<ipython-input-134-d648faf04e27>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  submit['predict'] = result


In [6]:
pd.read_csv('/kaggle/working/sumbmit_example.csv').head(10)

,inn_id,week,predict
0,inn1000051,118,59851.0
1,inn1000051,119,59851.0
2,inn1000051,120,59851.0
3,inn1000051,121,59851.0
4,inn1000051,122,59851.0
5,inn1000051,123,59851.0
6,inn1000051,124,59851.0
7,inn1000051,125,59851.0
8,inn1000051,126,59851.0
9,inn1000051,127,59851.0
